<a href="https://colab.research.google.com/github/mtsilimos/Codesnippets_Sentence-Embeddings-for-the-Classification-of-Explicit-Fine-grained-Discourse-Relations/blob/main/gridsearch_doc2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

df = pd.read_csv('/content/drive/My Drive/finalcorpus.csv', ';', encoding= 'unicode_escape')

df['text'].head()

<ipython-input-2-e599d8470047>:3: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  df = pd.read_csv('/content/drive/My Drive/finalcorpus.csv', ';', encoding= 'unicode_escape')


0    Once she asked for daddy, but went back to sle...
1    The medication they were giving her to clear u...
2        You can sit up, but you have to stay in there
3    She met his gaze defiantly for a moment longer...
4    If it's something we simply can't afford, that...
Name: text, dtype: object

In [3]:
def clean_text(text):
    text = text.lower()

    text = " ".join(text.split())
    return text


df['clean']= df['text'].map(clean_text)

In [4]:
df['text_clean'] = df['clean'].str.replace(r'[^\w\s]+', '')


<ipython-input-4-a45fbbb23007>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text_clean'] = df['clean'].str.replace(r'[^\w\s]+', '')


In [5]:
pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 4.9 MB/s eta 0:00:00


In [6]:
import contractions
df['cleaned'] = df['text_clean'].apply(lambda x: contractions.fix(x))

In [7]:
df.to_csv(r'C:\Users\mtsil\OneDrive\Documents\cleaneddoc.csv', ';')

In [8]:
df1 = pd.read_csv(r'C:\Users\mtsil\OneDrive\Documents\cleaneddoc.csv', ';', encoding= 'unicode_escape')

<ipython-input-8-43d737e86bb1>:1: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  df1 = pd.read_csv(r'C:\Users\mtsil\OneDrive\Documents\cleaneddoc.csv', ';', encoding= 'unicode_escape')


In [9]:
X = df['cleaned']
y = df['label']

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from gensim.parsing.preprocessing import preprocess_string
from sklearn import utils
from tqdm import tqdm

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
class Doc2VecModel(BaseEstimator):

    def __init__(self, learning_rate = 1):
        self.learning_rate = learning_rate

In [13]:
def fit(self, raw_documents, y=None):
        # Initialize model
        model = Doc2Vec(learning_rate=self.learning_rate)
        # Tag docs
        tagged_documents = []
        for index, row in raw_documents.iteritems():
            tag = '{}_{}'.format("type", index)
            tokens = row.split()
            tagged_documents.append(TaggedDocument(words=tokens, tags=[tag]))
            model.build_vocab(tagged_documents)
            model.train(tagged_documents, total_examples=len(tagged_documents), epochs=model.iter)

            return self

In [14]:
 def transform(self, raw_documents):
        X = []
        for index, row in raw_documents.iteritems():
            X.append(model.infer_vector(row))
        X = pd.DataFrame(X, index=raw_documents.index)
        return X

In [15]:
def fit_transform(self, raw_documents, y=None):
        self.fit(raw_documents)
        return self.transform(raw_documents)

In [16]:
param_grid = {'doc2vec__learning_rate': [0,1],


}

In [17]:
from sklearn.pipeline import make_pipeline, Pipeline
from imblearn.pipeline import Pipeline as imbPipeline

pl = imbPipeline([('doc2vec', Doc2VecModel()),('logistic', LogisticRegression())])

In [18]:
 from sklearn.model_selection import GridSearchCV
 gs_cv = GridSearchCV(estimator=pl, param_grid=param_grid, cv=5, n_jobs=-1,
 scoring="accuracy")

In [19]:
fitted = gs_cv.fit(X_train, y_train)

ValueError: ignored